<a href="https://colab.research.google.com/github/hejazn86/dogs_vs_cats_classifier/blob/master/main_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /drive


In [ ]:
import os


path = '/drive/My Drive/python/cats_and_dogs_filtered'
train_dir = os.path.join(path, 'train')
val_dir = os.path.join(path, 'validation')
train_dogs_dir = os.path.join(train_dir, 'dogs')
train_cats_dir = os.path.join(train_dir, 'cats')
val_dogs_dir = os.path.join(val_dir, 'dogs')
val_cats_dir = os.path.join(val_dir, 'cats')

#print(val_dogs_dir) 


In [ ]:
train_dogs_files = os.listdir(train_dogs_dir)
train_cats_files = os.listdir(train_cats_dir)
val_dogs_files = os.listdir(val_dogs_dir)
val_cats_files = os.listdir(val_cats_dir)


In [ ]:
import tensorflow as tf

classifier = tf.keras.models.Sequential([                                         
tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu', padding = 'same', input_shape = (150, 150, 3)),
tf.keras.layers.MaxPool2D(pool_size = (2, 2), strides = (2, 2)),
#tf.keras.layers.Dropout(0.2),
tf.keras.layers.Conv2D(128, (3, 3), activation = 'relu', padding = 'same'),
tf.keras.layers.MaxPool2D(pool_size = (2, 2), strides = (2, 2)),
#tf.keras.layers.Dropout(0.2),
tf.keras.layers.Conv2D(128, (3, 3), activation = 'relu', padding = 'same'),
tf.keras.layers.MaxPool2D(pool_size = (3, 3), strides = (2, 2)),
#tf.keras.layers.Dropout(0.2),
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(512, activation = 'relu'),
tf.keras.layers.Dropout(0.7), 
tf.keras.layers.Dense(1, activation = 'sigmoid')                                         
])

classifier.compile(optimizer = tf.keras.optimizers.Adam(lr=1e-4), loss = 'binary_crossentropy', metrics = ['acc'])

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_data_generator = ImageDataGenerator(rescale = 1/225)
val_data_generator = ImageDataGenerator(rescale = 1/255)

train_generator = train_data_generator.flow_from_directory(train_dir,
                                                           target_size = (150, 150),
                                                           batch_size = 100,
                                                           class_mode = 'binary')
val_generator = val_data_generator.flow_from_directory(val_dir,
                                                       target_size = (150, 150),
                                                       batch_size = 50,
                                                       class_mode = 'binary')

history = classifier.fit(train_generator,
                         validation_data = val_generator,
                         steps_per_epoch = 100,
                         epochs = 30,
                         verbose = 1,
                         validation_steps = 50)

print(train_generator[20].shape)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy  as np

train_data_generator = ImageDataGenerator(rescale= 1.0/255.0)

val_data_generator = ImageDataGenerator(rescale= 1.0/ 255.0)


train_generator = train_data_generator.flow_from_directory(train_dir,
                                                           target_size = (150, 150),
                                                           batch_size = 40,
                                                           class_mode = 'binary')
val_generator = val_data_generator.flow_from_directory(val_dir,
                                                       target_size = (150, 150),
                                                       batch_size = 20,
                                                       class_mode = 'binary')

print(train_generator.dtype)

history = classifier.fit_generator(train_generator, steps_per_epoch = 50, epochs = 100, validation_data = val_generator, validation_steps = 50)

classifier.summary()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

predicted = classifier.predict(val_generator)
val_loss, val_acc = classifier.evaluate(val_generator, steps = 5)

epochs = range(len(history.history['acc']))

plt.figure(figsize = (12, 7))
plt.subplot(121).plot(epochs, history.history['loss'], 'b*', label = 'Train_loss')
plt.subplot(121).plot(epochs, history.history['val_loss'], 'r', label = 'Test_loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.grid(True)
plt.legend()

plt.subplot(122).plot(epochs, history.history['acc'], 'b*', label = 'Train_accuracy')
plt.subplot(122).plot(epochs, history.history['val_acc'], 'r', label = 'Test_accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('acc')
plt.grid(True)
plt.legend(['Train', 'Test'], loc = 'uper right')
plt.show()

In [ ]:
import os
import signal
 
os.kill( os.getpid(), signal.SIGKILL)